# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### *Task 1- Creating DataFrame as given in Question*

***Lets first import all the neccessary libraries***

In [23]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('All the above Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

All the above Libraries imported.


***Scraping wiki page***


In [24]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [25]:
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_postal_codes = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_postal_codes.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_postal_codes.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


***Droping rows with Borough-Not assigned***

In [26]:
df_postal_codes['Borough'].replace('Not assigned', np.nan, inplace=True)
df_postal_codes.dropna(subset=['Borough'], inplace=True)

df_postal_codes.head()




,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



***Combining two rows with same Borough and different neighbourhood to one row where different neighbours will be seperated using ','.***



In [27]:
df_postal_codes_final = df_postal_codes.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_postal_codes_final.columns = ['Postcode', 'Borough', 'Neighbourhood']
df_postal_codes_final



,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"



***Check if there are any Neighbourhood with 'Not assigned' value.***

In [28]:
df1 = df_postal_codes_final['Neighbourhood'].str.contains('Not assigned') 
df1

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

In [29]:
print("Shape of the DataFrame:",df_postal_codes_final.shape)

Shape of the DataFrame: (103, 3)


### *Task_2*

In [30]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']

In [31]:
df_pos = pd.merge(df_postal_codes_final, df_geo, on=['Postcode'], how='inner')


df_tor = df_pos[['Postcode','Borough', 'Neighbourhood',  'Latitude', 'Longitude']].copy()

df_tor.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
df_tor

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


***Latitude and LOngitude of Toronto***

In [33]:
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="Course-capstone-project")
location = geolocator.geocode('Toronto, Canada')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [34]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'], df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6).add_to(map_toronto)  
    
map_toronto

In [39]:
borough_names = list(df_tor.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [42]:
df_tor = df_tor[df_tor['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_tor.shape)
df_tor.head()

(39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [45]:

# add markers to map
for lat, lng, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Borough'],df_tor['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [49]:
CLIENT_ID = 'KEIL1ALZLAGPVQFPBGX2ZOYVMDBNS2ME4PRREGL52YD01IKN' # your Foursquare ID
CLIENT_SECRET = 'PEGNKRCSTGQCKYP0TAVAGFIT5LPKOJM12FR25UZ3QJ3BWJYY'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KEIL1ALZLAGPVQFPBGX2ZOYVMDBNS2ME4PRREGL52YD01IKN
CLIENT_SECRET:PEGNKRCSTGQCKYP0TAVAGFIT5LPKOJM12FR25UZ3QJ3BWJYY


In [52]:
radius = 500
LIMIT = 400

venues = []

for lat, long, post, borough, neighborhood in zip(df_tor['Latitude'], df_tor['Longitude'], df_tor['Postcode'], df_tor['Borough'], 
                                                  df_tor['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=KEIL1ALZLAGPVQFPBGX2ZOYVMDBNS2ME4PRREGL52YD01IKN&client_secret=PEGNKRCSTGQCKYP0TAVAGFIT5LPKOJM12FR25UZ3QJ3BWJYY&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=400".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?client_id=KEIL1ALZLAGPVQFPBGX2ZOYVMDBNS2ME4PRREGL52YD01IKN&client_secret=PEGNKRCSTGQCKYP0TAVAGFIT5LPKOJM12FR25UZ3QJ3BWJYY&v=20180605%20%20%20%20%20%20&ll=43.653963,-79.387207&radius=500&limit=400 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000000780C4D8160>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [53]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 9 elements

In [ ]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()